In [1]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=be3a725875f43fbccd655c81ed4fe91da9451b19f0421397489b0fac3b45d836
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=9us2fjunOMdAueINh9JnfKiasS1pRV&prompt=consent&token_usage=remote&access_type=offline&code_challenge=U_PElEKuZIL_uyZVtK8W69e81GL6PCDPZEMBo9V_Kks&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrAxEr3y6y27SFxfLhEIvetiWtzIjTNK4RLm8WO4MhbAj7G2jqh7ibPMWb_D4zFgmw

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [3]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [4]:
client = bigquery.Client(project='repositoriodedadosgpsp')
query = "SELECT ano, tipo_ocupacao,sigla_uf,id_municipio, sum(n_profissionais) as quantidade_vinculos  FROM `repositoriodedadosgpsp.Datalake.CNES_vinculo_saude_tipo_ocupacao_v2` where publico_privado='Público' group by ano, tipo_ocupacao, sigla_uf,id_municipio"
df = client.query(query).to_dataframe()
df

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos
0,2008,Técnicos e auxiliares de enfermagem,AP,1600600,225
1,2008,Trabalhadores em serviços de promoção e apoio ...,RR,1400407,28
2,2008,Nutricionistas,AP,1600303,31
3,2008,Agentes da saúde e do meio ambiente,AP,1600600,12
4,2008,Trabalhadores de moldagem de metais e de ligas...,RR,1400100,2
...,...,...,...,...,...
1836885,2024,Assistentes sociais e economistas domésticos,TO,1718501,1
1836886,2024,Técnicos e auxiliares de enfermagem,TO,1718501,12
1836887,2024,"Agentes, assistentes e auxiliares administrativos",TO,1720655,1
1836888,2024,Psicólogos e psicanalistas,TO,1703883,1


In [5]:
df['tipo_2'] = np.where(df['tipo_ocupacao'].str.startswith('Enfermeiros e afins'),'Enfermeiros e afins',df['tipo_ocupacao'])


In [6]:
df

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
0,2008,Técnicos e auxiliares de enfermagem,AP,1600600,225,Técnicos e auxiliares de enfermagem
1,2008,Trabalhadores em serviços de promoção e apoio ...,RR,1400407,28,Trabalhadores em serviços de promoção e apoio ...
2,2008,Nutricionistas,AP,1600303,31,Nutricionistas
3,2008,Agentes da saúde e do meio ambiente,AP,1600600,12,Agentes da saúde e do meio ambiente
4,2008,Trabalhadores de moldagem de metais e de ligas...,RR,1400100,2,Trabalhadores de moldagem de metais e de ligas...
...,...,...,...,...,...,...
1836885,2024,Assistentes sociais e economistas domésticos,TO,1718501,1,Assistentes sociais e economistas domésticos
1836886,2024,Técnicos e auxiliares de enfermagem,TO,1718501,12,Técnicos e auxiliares de enfermagem
1836887,2024,"Agentes, assistentes e auxiliares administrativos",TO,1720655,1,"Agentes, assistentes e auxiliares administrativos"
1836888,2024,Psicólogos e psicanalistas,TO,1703883,1,Psicólogos e psicanalistas


In [7]:
df = df[(df['tipo_2']=='Enfermeiros e afins') & (df['ano']>2017) & (df['ano']<=2023)]

In [8]:
df

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
710910,2018,Enfermeiros e afins,RR,1400456,38,Enfermeiros e afins
710993,2018,Enfermeiros e afins,AC,1200203,160,Enfermeiros e afins
711016,2018,Enfermeiros e afins,AL,2702702,2,Enfermeiros e afins
711018,2018,Enfermeiros e afins,AL,2701100,6,Enfermeiros e afins
711056,2018,Enfermeiros e afins,AL,2703007,11,Enfermeiros e afins
...,...,...,...,...,...,...
1752334,2023,Enfermeiros e afins,TO,1707405,8,Enfermeiros e afins
1752369,2023,Enfermeiros e afins,TO,1703909,6,Enfermeiros e afins
1752378,2023,Enfermeiros e afins,TO,1707108,9,Enfermeiros e afins
1752393,2023,Enfermeiros e afins,TO,1714302,3,Enfermeiros e afins


In [10]:
## Chamando a quantidade de população.
## Tabela no BQ equivalente ao CSV: repositoriodedadosgpsp.Datalake.IBGE_populacao_censo_2022

pop = pd.read_csv("POP_CD2022_ATT.csv");
pop

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POP. COLETADA,POP. IMPUTADA,POPULAÇÃO,id_municipio
0,RO,11,15,Alta Floresta D'Oeste,20865,629,21494,1100015
1,RO,11,23,Ariquemes,93006,3827,96833,1100023
2,RO,11,31,Cabixi,5100,251,5351,1100031
3,RO,11,49,Cacoal,84923,1964,86887,1100049
4,RO,11,56,Cerejeiras,15297,593,15890,1100056
...,...,...,...,...,...,...,...,...
5565,GO,52,22005,Vianópolis,14345,611,14956,5222005
5566,GO,52,22054,Vicentinópolis,8592,176,8768,5222054
5567,GO,52,22203,Vila Boa,4168,47,4215,5222203
5568,GO,52,22302,Vila Propício,5594,221,5815,5222302


In [11]:
df.head(20)

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
710910,2018,Enfermeiros e afins,RR,1400456,38,Enfermeiros e afins
710993,2018,Enfermeiros e afins,AC,1200203,160,Enfermeiros e afins
711016,2018,Enfermeiros e afins,AL,2702702,2,Enfermeiros e afins
711018,2018,Enfermeiros e afins,AL,2701100,6,Enfermeiros e afins
711056,2018,Enfermeiros e afins,AL,2703007,11,Enfermeiros e afins
711058,2018,Enfermeiros e afins,AL,2700508,12,Enfermeiros e afins
711074,2018,Enfermeiros e afins,AL,2707602,9,Enfermeiros e afins
711112,2018,Enfermeiros e afins,AL,2705507,28,Enfermeiros e afins
711115,2018,Enfermeiros e afins,AL,2700409,30,Enfermeiros e afins
711142,2018,Enfermeiros e afins,AL,2706307,52,Enfermeiros e afins


In [12]:
df['id_municipio'] = pd.to_numeric(df['id_municipio'])

<ipython-input-12-a8331f5d24a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id_municipio'] = pd.to_numeric(df['id_municipio'])


In [13]:
x=df.pivot_table(index=['ano','sigla_uf','id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)
y = pd.DataFrame(x)
df = y.reset_index()

<ipython-input-13-d833fc746e54>:1: FutureWarning: The provided callable <function sum at 0x796a63516dd0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  x=df.pivot_table(index=['ano','sigla_uf','id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)


In [14]:
df1= df.merge(pop[['UF','NOME DO MUNICÍPIO','id_municipio','POPULAÇÃO']], left_on='id_municipio',right_on='id_municipio').sort_values('quantidade_vinculos', ascending=False)

In [15]:
df1['taxa'] = df1['quantidade_vinculos']/df1['POPULAÇÃO']*1000

In [16]:
sum(df1[df1['ano']==2023]['POPULAÇÃO'])

203080756

In [17]:
sum(df1[df1['ano']==2023]['quantidade_vinculos'])/sum(df1[df1['ano']==2023]['POPULAÇÃO'])*1000

1.4911604918390198

In [18]:
## validando alguns dados
df1[(df1['ano']==2023) & (df1['UF']=='RJ')].sort_values('taxa', ascending=False).head(30)

,ano,sigla_uf,id_municipio,tipo_2,quantidade_vinculos,UF,NOME DO MUNICÍPIO,POPULAÇÃO,taxa
22107,2023,RJ,3306305,Enfermeiros e afins,811,RJ,Volta Redonda,261563,3.100591
21801,2023,RJ,3302700,Enfermeiros e afins,589,RJ,Maricá,197277,2.98565
21951,2023,RJ,3304508,Enfermeiros e afins,26,RJ,Rio das Flores,8954,2.90373
21843,2023,RJ,3303302,Enfermeiros e afins,1387,RJ,Niterói,481749,2.879093
21561,2023,RJ,3300100,Enfermeiros e afins,468,RJ,Angra dos Reis,167434,2.795131
21795,2023,RJ,3302601,Enfermeiros e afins,115,RJ,Mangaratiba,41220,2.789908
21783,2023,RJ,3302452,Enfermeiros e afins,15,RJ,Macuco,5415,2.770083
21639,2023,RJ,3300902,Enfermeiros e afins,38,RJ,Cambuci,14616,2.599891
22041,2023,RJ,3305505,Enfermeiros e afins,227,RJ,Saquarema,89559,2.534642
21963,2023,RJ,3304557,Enfermeiros e afins,15653,RJ,Rio de Janeiro,6211223,2.520116


In [19]:
df1.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
       'UF', 'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa'],
      dtype='object')

In [20]:
## Renomear colunas

df = df1[['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
        'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa']].rename(columns={'tipo_2':'tipo_ocupacao','UF':'sigla_uf', 'NOME DO MUNICÍPIO':'nome_municipio','POPULAÇÃO':'populacao_domiciliada' })

In [21]:
df

,ano,sigla_uf,id_municipio,tipo_ocupacao,quantidade_vinculos,nome_municipio,populacao_domiciliada,taxa
32087,2023,SP,3550308,Enfermeiros e afins,15886,São Paulo,11451999,1.387181
32086,2022,SP,3550308,Enfermeiros e afins,15690,São Paulo,11451999,1.370066
21963,2023,RJ,3304557,Enfermeiros e afins,15653,Rio de Janeiro,6211223,2.520116
21962,2022,RJ,3304557,Enfermeiros e afins,14601,Rio de Janeiro,6211223,2.350745
32085,2021,SP,3550308,Enfermeiros e afins,14210,São Paulo,11451999,1.240831
...,...,...,...,...,...,...,...,...
11982,2018,MG,3156205,Enfermeiros e afins,1,Rochedo de Minas,2291,0.436491
23919,2018,RS,4303673,Enfermeiros e afins,1,Campestre da Serra,3242,0.308452
23920,2019,RS,4303673,Enfermeiros e afins,1,Campestre da Serra,3242,0.308452
23921,2020,RS,4303673,Enfermeiros e afins,1,Campestre da Serra,3242,0.308452


In [22]:
df= df[['ano', 'sigla_uf', 'nome_municipio','id_municipio','populacao_domiciliada', 'tipo_ocupacao',
       'quantidade_vinculos',
       'taxa']]

In [23]:
df

,ano,sigla_uf,nome_municipio,id_municipio,populacao_domiciliada,tipo_ocupacao,quantidade_vinculos,taxa
32087,2023,SP,São Paulo,3550308,11451999,Enfermeiros e afins,15886,1.387181
32086,2022,SP,São Paulo,3550308,11451999,Enfermeiros e afins,15690,1.370066
21963,2023,RJ,Rio de Janeiro,3304557,6211223,Enfermeiros e afins,15653,2.520116
21962,2022,RJ,Rio de Janeiro,3304557,6211223,Enfermeiros e afins,14601,2.350745
32085,2021,SP,São Paulo,3550308,11451999,Enfermeiros e afins,14210,1.240831
...,...,...,...,...,...,...,...,...
11982,2018,MG,Rochedo de Minas,3156205,2291,Enfermeiros e afins,1,0.436491
23919,2018,RS,Campestre da Serra,4303673,3242,Enfermeiros e afins,1,0.308452
23920,2019,RS,Campestre da Serra,4303673,3242,Enfermeiros e afins,1,0.308452
23921,2020,RS,Campestre da Serra,4303673,3242,Enfermeiros e afins,1,0.308452


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33414 entries, 32087 to 20525
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ano                    33414 non-null  Int64  
 1   sigla_uf               33414 non-null  object 
 2   nome_municipio         33414 non-null  object 
 3   id_municipio           33414 non-null  int64  
 4   populacao_domiciliada  33414 non-null  int64  
 5   tipo_ocupacao          33414 non-null  object 
 6   quantidade_vinculos    33414 non-null  Int64  
 7   taxa                   33414 non-null  Float64
dtypes: Float64(1), Int64(2), int64(2), object(3)
memory usage: 2.4+ MB


Subindo para o GBQ

In [26]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),
 bigquery.SchemaField('id_municipio','FLOAT',description='Identificador do município pelo IBGE'),
 bigquery.SchemaField('populacao_domiciliada','FLOAT',description='População domiciliada (prévia do Censo 2022)'),
 bigquery.SchemaField('tipo_ocupacao','STRING',description='Qual a ocupação daquele vínculo'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('taxa','Float',description='Taxa de enfermeiros por 1000 habitantes')
 ]

In [27]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [28]:
table_ref = dataset_ref.table('CNES_enfermeiros_mil_habitantes_v2') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=d048d069-a6e6-49f5-9f02-115c76012d14>